In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import pickle  
import os
from pycaret.classification import load_model, predict_model
import datetime
import numpy as np
from scipy.stats import entropy

setup_function():
- load out-of-sample data
- load all models

In [ ]:
#Loading different datasets
#df = pd.read_excel('/home/dsg/VORTEX_FINAL/PRODUCTION/DATA/real_world/real_world_data.xlsx',sheet_name='quiruelas', engine='openpyxl')
#df = pd.read_excel('/home/dsg/VORTEX_FINAL/PRODUCTION/DATA/real_world/real_world_data.xlsx',sheet_name='v_higueras', engine='openpyxl')
#df = pd.read_excel('/home/dsg/VORTEX_FINAL/PRODUCTION/DATA/real_world/real_world_data.xlsx',sheet_name='Alberite', engine='openpyxl')
#df = pd.read_excel('/home/dsg/VORTEX_FINAL/PRODUCTION/DATA/real_world/real_world_data.xlsx',sheet_name='Paternanbidea', engine='openpyxl')
#df = pd.read_excel('/home/dsg/VORTEX_FINAL/PRODUCTION/DATA/real_world/real_world_data.xlsx',sheet_name='Can_Gambus', engine='openpyxl')
#df = pd.read_excel('/home/dsg/VORTEX_FINAL/PRODUCTION/DATA/real_world/real_world_data.xlsx',sheet_name='CatalonianSites', engine='openpyxl')
df.info()


In [ ]:
#Loading models
full_model = load_model(model_path)
destilled_model = load_model(model_path)
#specific
VdH = load_model(model_path)
PQ = load_model(model_path)
French = load_model(model_path)

### Prediction_function():
- 

In [ ]:

def prediction(model_path, excel_path, exclude_columns=None, output_column='predictions'):
    """
    Carga un archivo Excel,
    hace predicciones usando un modelo pre-entrenado,
    y guarda los resultados en un nuevo archivo Excel o CSV.

    :param model_path: Ruta al archivo del modelo 
    :param excel_path: Ruta al archivo con los datos de entrada
    :param exclude_columns: Lista de columnas a excluir como entrada (opcional)
    :param output_column: Nombre de la columna para las predicciones (por defecto: 'predictions')
    :return: Ruta al archivo de salida con las predicciones
    """
    # Loading  the model usando PyCaret
    modelo = load_model(model_path)

    # Cargar los datos
    df = pd.read_excel(excel_path)


    # Excluir columnas si se especifican
    if exclude_columns:
        df = df.drop(columns=[col for col in exclude_columns], errors='ignore')

    # Hacer predicciones usando PyCaret
    predictions = predict_model(model, data=df,raw_score=True)
    
    # El resultado de predict_model ya incluye las predicciones, 
    # generalmente en una columna llamada 'Label' o similar
    # Renombrar la columna de predicciones si es necesario
    if 'Label' in predicciones.columns:
        predicciones = predicciones.rename(columns={'Label': output_column})
    elif 'prediction_label' in predicciones.columns:
        predicciones = predicciones.rename(columns={'prediction_label': output_column})
    else:
        # Si la columna de predicciones tiene otro nombre, ajusta esto según sea necesario
        pass

   
    # Create name for the output file
    base_name = os.path.splitext(excel_path)[0]
    current_date = datetime.datetime.now().strftime("%Y%m%d")
    output_path = f"{base_name}_{current_date}_predictions.xlsx"
   

    # save results
    predictions.to_excel(output_path, index=False)
    predictions.to_csv(output_path, index=False)

    return output_path

In [ ]:

# Usage:
model_path = '20240827_SinCaDestilledVortex2SMOTECalibrated'
excel_path = "./DATA/VARISCITA ROMANA100.xlsx"
exclude_columns = [] 

try:
    output_file = predict_from_excel(model_path, excel_path, exclude_columns)
    print(f"Predictions saved in: {output_file}")
except Exception as e:
    print(f"Error processing the file: {str(e)}")


class ProvenanceAndUncertaintyAnalysis:

- def predictions_uncertain analysis():
  - Confidence treshold >0.7
  - Average confidence per site
  - Median entropy per site
  - Median probabilities per site

- def provenance_determination():
  - Majority vote sourcing
  - Homogeneity
  - summary table
    - Site
    - Samples Analyzed
    - Samples sourced to Gavá
    - Samples sourced to Aliste
    - Samples sourced to Encinasola
    - Median entropy per site
    - (%) of uncertain samples
    - number of samples used for sourcing
    - Majority vote sourcing
    - Homogenity

 


 

In [26]:


def predictions_uncertainty_analysis(prediction_df, confidence_threshold=0.7):


    """
    Performs uncertainty analysis per site

    Args:
        prediction_df: df with probabilistic predictions
        confidence_treshold: threshold to mark the prediction as uncertain
     
    """
    # Convert score columns to numbers
    score_cols = ['prediction_score_CT', 'prediction_score_PCM', 'prediction_score_PDLC']
    for col in score_cols:
        df[col] = df[col].str.replace(',', '.').astype(float)
    
    # Obtaining probabilities
    probas = df[score_cols].values
    
    # Obtain prediction labels and confidence
    predictions = df['predicciones'].values
    confidences = np.max(probas, axis=1)
    
    # Marking predictions below the threshold as uncertain
    uncertain_mask = confidences < confidence_threshold
    predictions_with_uncertainty = predictions.copy()
    predictions_with_uncertainty[uncertain_mask] = 'uncertain'
    
    # Calculate entropy 
    entropies = np.array([entropy(probs, base=2) for probs in probas])
    
    # Create DataFrame with results
    results_df = pd.DataFrame({
        'id': df['id'],
        'Site': df['Site'],
        'Original_predictions': predictions,
        'prediction_score_CT': df['prediction_score_CT'],
        'prediction_score_PCM':df['prediction_score_PCM'],
        'prediction_score_PDLC': df['prediction_score_PDLC'],
        'Uncertainty_treshold_predictions': predictions_with_uncertainty,
        'entropy': entropies
    })
    
    # Print basic metrics
    n_uncertain = np.sum(uncertain_mask)
    print(f"Uncertain predictions: {n_uncertain}/{len(df)} ({(n_uncertain/len(df)*100):.1f}%)")
    print(f"Mean dataset entropy: {entropies.mean():.3f}")
    
    
    # Calculate and display median entropy per site
    print("\nMedian entropy per site:")
    entropy_median_by_site = results_df.groupby('Site')['entropy'].median()
    for site, median in entropy_median_by_site.items():
        print(f"{site}: {median:.3f}")
    
    return results_df

In [27]:
# Cargar y procesar datos
df = pd.read_csv('/home/dsg/VORTEX_FINAL/PRODUCTION/DATA/20241118_PrediccionesVortexTodas.csv',encoding='latin-1')
results = predictions_uncertainty_analysis(df, confidence_threshold=0.7)

# Mostrar primeras filas de resultados
print("\nPrimeras 5 filas de resultados:")
results.head()

Uncertain predictions: 147/576 (25.5%)
Mean dataset entropy: 0.794

Median entropy per site:
Auverné: 0.710
Can Gambús I: 0.233
Can Sadurni: 1.489
Cova Cassimanya: 1.409
Dolmen de Alberite: 0.343
Josseliére: 0.710
Kervilor: 0.710
La serreta: 0.882
Luffang: 0.721
Paternanbidea: 0.555
Peñas Quiruelas: 0.757
Plichancourt: 0.738
Roca de L'ivet: 1.462
Tumulus St Michel: 0.710
Valle de las Higueras: 0.876

Primeras 5 filas de resultados:


,id,Site,Original_predictions,prediction_score_CT,prediction_score_PCM,prediction_score_PDLC,Uncertainty_treshold_predictions,entropy
0,MP-865,Can Sadurni,CT,0.72,0.13,0.15,CT,1.134419
1,MP-824,Roca de L'ivet,PDLC,0.22,0.38,0.40,uncertain,1.539798
2,MP-795,Can Sadurni,CT,0.48,0.33,0.19,uncertain,1.491318
3,MP-797,Can Sadurni,PDLC,0.19,0.34,0.47,uncertain,1.496356
4,MP-823,Roca de L'ivet,PDLC,0.22,0.34,0.44,uncertain,1.530894


In [50]:
def provenance_determination(df):

     # Convert score columns to numbers
    score_columns = ['prediction_score_CT', 'prediction_score_PCM', 'prediction_score_PDLC']
    #for col in score_cols:
        #df[col] = df[col].str.replace(',', '.').astype(float)
    
    df['max_prob'] = df[score_columns].max(axis=1)
    
    results = []
    for site in df['Site'].unique():
        site_data = df[df['Site'] == site]
        #n_uncertain = sum(site_data['max_prob'] < 0.70)
        avg_uncertain = sum(site_data['max_prob'] < 0.70)/ len(site_data)*100
        high_conf = site_data[site_data['max_prob'] > 0.70]
        median_entropy = site_data.entropy.median()
        
        if len(high_conf) > 0:
            consensus = high_conf['Uncertainty_treshold_predictions'].mode().iloc[0] # majority vote as mode of high confidence samples
            consistency = sum(high_conf['Uncertainty_treshold_predictions'] == consensus) / len(high_conf)
            n_consensus_pred = len(high_conf)  # Número de predicciones usadas
        else:
            consensus = 'No consensus'
            consistency = 0
            n_consensus_pred = 0
            
        results.append({
            'Site': site,
            'Samples_analyzed': len(site_data),
            'Gavá': len(site_data[site_data['Original_predictions'] == 'CT']) ,
            'Encinasola': len(site_data[site_data['Original_predictions'] == 'PCM']),
            'Aliste': len(site_data[site_data['Original_predictions'] == 'PDLC']),
            'Uncertain(%)': round(avg_uncertain),
            'Samples_for_provenance': n_consensus_pred,
            'Median_entropy': round(median_entropy,2),
            'Consensus': consensus,
            'Homogeneity': round(consistency,2)
        })
    
    return pd.DataFrame(results)

In [51]:
summary_df = provenance_determination(results)
summary_df

,Site,Samples_analyzed,Gavá,Encinasola,Aliste,Uncertain(%),Samples_for_provenance,Median_entropy,Consensus,Homogeneity
0,Can Sadurni,8,6,0,2,75,2,1.49,CT,1.00
1,Roca de L'ivet,16,1,3,12,94,1,1.46,PDLC,1.00
2,La serreta,59,58,0,1,27,42,0.88,CT,1.00
3,Cova Cassimanya,12,10,0,2,75,3,1.41,CT,1.00
4,Valle de las Higueras,276,14,21,241,31,191,0.88,PDLC,0.97
5,Peñas Quiruelas,52,0,5,47,19,42,0.76,PDLC,0.98
6,Luffang,9,0,0,9,0,9,0.72,PDLC,1.00
7,Tumulus St Michel,3,0,0,3,0,3,0.71,PDLC,1.00
8,Auverné,3,0,0,3,0,3,0.71,PDLC,1.00
9,Josseliére,9,0,0,9,0,9,0.71,PDLC,1.00


Plotting function:
- Create uncertain plot
- Save

In [ ]:
def plot_function():